In [8]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
import tensorflow as tf

In [4]:

# Define the directory paths
normal_dir = 'chest_xray/train/NORMAL'
pneumonia_dir = 'chest_xray/train/PNEUMONIA'

# Get the list of all files in each directory
normal_files = [os.path.join(normal_dir, file) for file in os.listdir(normal_dir)]
pneumonia_files = [os.path.join(pneumonia_dir, file) for file in os.listdir(pneumonia_dir)]

# Create a DataFrame
train_df = pd.DataFrame({
    'file': normal_files + pneumonia_files,
    'label': ['normal'] * len(normal_files) + ['pneumonia'] * len(pneumonia_files)
})

print(train_df)

                                                   file      label
0     chest_xray/train/NORMAL/NORMAL2-IM-0927-0001.jpeg     normal
1     chest_xray/train/NORMAL/NORMAL2-IM-1056-0001.jpeg     normal
2             chest_xray/train/NORMAL/IM-0427-0001.jpeg     normal
3     chest_xray/train/NORMAL/NORMAL2-IM-1260-0001.jpeg     normal
4        chest_xray/train/NORMAL/IM-0656-0001-0001.jpeg     normal
...                                                 ...        ...
5213  chest_xray/train/PNEUMONIA/person142_virus_288...  pneumonia
5214  chest_xray/train/PNEUMONIA/person364_bacteria_...  pneumonia
5215  chest_xray/train/PNEUMONIA/person1323_virus_22...  pneumonia
5216  chest_xray/train/PNEUMONIA/person772_virus_140...  pneumonia
5217  chest_xray/train/PNEUMONIA/person501_virus_101...  pneumonia

[5218 rows x 2 columns]


In [10]:
# Define the directory paths
normal_dir = 'chest_xray/test/NORMAL'
pneumonia_dir = 'chest_xray/test/PNEUMONIA'

# Get the list of all files in each directory
normal_files = [os.path.join(normal_dir, file) for file in os.listdir(normal_dir)]
pneumonia_files = [os.path.join(pneumonia_dir, file) for file in os.listdir(pneumonia_dir)]

# Create a DataFrame
test_df = pd.DataFrame({
    'file': normal_files + pneumonia_files,
    'label': ['normal'] * len(normal_files) + ['pneumonia'] * len(pneumonia_files)
})

print(test_df)

                                                  file      label
0             chest_xray/test/NORMAL/IM-0031-0001.jpeg     normal
1             chest_xray/test/NORMAL/IM-0025-0001.jpeg     normal
2     chest_xray/test/NORMAL/NORMAL2-IM-0272-0001.jpeg     normal
3     chest_xray/test/NORMAL/NORMAL2-IM-0102-0001.jpeg     normal
4     chest_xray/test/NORMAL/NORMAL2-IM-0229-0001.jpeg     normal
..                                                 ...        ...
619  chest_xray/test/PNEUMONIA/person120_bacteria_5...  pneumonia
620  chest_xray/test/PNEUMONIA/person171_bacteria_8...  pneumonia
621  chest_xray/test/PNEUMONIA/person109_bacteria_5...  pneumonia
622  chest_xray/test/PNEUMONIA/person83_bacteria_41...  pneumonia
623  chest_xray/test/PNEUMONIA/person112_bacteria_5...  pneumonia

[624 rows x 2 columns]


In [11]:
# Define the directory paths
normal_dir = 'chest_xray/val/NORMAL'
pneumonia_dir = 'chest_xray/val/PNEUMONIA'

# Get the list of all files in each directory
normal_files = [os.path.join(normal_dir, file) for file in os.listdir(normal_dir)]
pneumonia_files = [os.path.join(pneumonia_dir, file) for file in os.listdir(pneumonia_dir)]

# Create a DataFrame
val_df = pd.DataFrame({
    'file': normal_files + pneumonia_files,
    'label': ['normal'] * len(normal_files) + ['pneumonia'] * len(pneumonia_files)
})

print(val_df)

                                                 file      label
0     chest_xray/val/NORMAL/NORMAL2-IM-1440-0001.jpeg     normal
1     chest_xray/val/NORMAL/NORMAL2-IM-1437-0001.jpeg     normal
2                     chest_xray/val/NORMAL/.DS_Store     normal
3     chest_xray/val/NORMAL/NORMAL2-IM-1431-0001.jpeg     normal
4     chest_xray/val/NORMAL/NORMAL2-IM-1436-0001.jpeg     normal
5     chest_xray/val/NORMAL/NORMAL2-IM-1430-0001.jpeg     normal
6     chest_xray/val/NORMAL/NORMAL2-IM-1438-0001.jpeg     normal
7     chest_xray/val/NORMAL/NORMAL2-IM-1442-0001.jpeg     normal
8     chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg     normal
9                  chest_xray/val/PNEUMONIA/.DS_Store  pneumonia
10  chest_xray/val/PNEUMONIA/person1950_bacteria_4...  pneumonia
11  chest_xray/val/PNEUMONIA/person1951_bacteria_4...  pneumonia
12  chest_xray/val/PNEUMONIA/person1952_bacteria_4...  pneumonia
13  chest_xray/val/PNEUMONIA/person1946_bacteria_4...  pneumonia
14  chest_xray/val/PNEUMO

Générateur des images

In [12]:
# Initialize the ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255.,  # scale pixel values to [0, 1]
    shear_range=0.2,  # randomly apply shearing transformations
    zoom_range=0.2,  # randomly zooming inside pictures
    horizontal_flip=True  # randomly flip half of the images horizontally
)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="file",
    y_col="label",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="file",
    y_col="label",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="file",
    y_col="label",
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')


Found 5216 validated image filenames belonging to 2 classes.
Found 16 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


/Users/titouanlegourrierec/anaconda3/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 2 invalid image filename(s) in x_col="file". These filename(s) will be ignored.
  warnings.warn(
/Users/titouanlegourrierec/anaconda3/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 2 invalid image filename(s) in x_col="file". These filename(s) will be ignored.
  warnings.warn(


Création du modèle

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
              metrics=['accuracy'])

Entraînement du modèle

In [14]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # nombre d'images = batch_size * steps
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,  # nombre d'images = batch_size * steps
      verbose=2)

Epoch 1/15


100/100 - 28s - loss: 0.6646 - accuracy: 0.7559 - val_loss: 1.5724 - val_accuracy: 0.5000 - 28s/epoch - 278ms/step
Epoch 2/15
100/100 - 28s - loss: 0.3611 - accuracy: 0.8503 - 28s/epoch - 277ms/step
Epoch 3/15
100/100 - 28s - loss: 0.2772 - accuracy: 0.8913 - 28s/epoch - 275ms/step
Epoch 4/15
100/100 - 29s - loss: 0.2510 - accuracy: 0.8972 - 29s/epoch - 288ms/step
Epoch 5/15
100/100 - 29s - loss: 0.2283 - accuracy: 0.9103 - 29s/epoch - 295ms/step
Epoch 6/15
100/100 - 29s - loss: 0.1952 - accuracy: 0.9209 - 29s/epoch - 291ms/step
Epoch 7/15
100/100 - 29s - loss: 0.1827 - accuracy: 0.9297 - 29s/epoch - 290ms/step
Epoch 8/15
100/100 - 29s - loss: 0.1825 - accuracy: 0.9319 - 29s/epoch - 287ms/step
Epoch 9/15
100/100 - 29s - loss: 0.1717 - accuracy: 0.9325 - 29s/epoch - 287ms/step
Epoch 10/15
100/100 - 30s - loss: 0.1622 - accuracy: 0.9409 - 30s/epoch - 297ms/step
Epoch 11/15
100/100 - 30s - loss: 0.1433 - accuracy: 0.9466 - 30s/epoch - 298ms/step
Epoch 12/15
100/100 - 29s - loss: 0.1394 - 

In [15]:
model.evaluate(test_generator)

20/20 [==============================] - 4s 221ms/step - loss: 0.5625 - accuracy: 0.7821


[0.5625074505805969, 0.7820512652397156]

In [13]:
#test
print("hello world")

hello world
